# MULTIMEDIA 
## PROJECT 1: Image Compression
### B.Sc. in Informatics Engineering, FCTUC

Authors: 
 - André Carvalho, no. 2019216156
 - Paulo Cortesão, no. 2019216517

In [ ]:
# code imports
import matplotlib
import os
from scipy.fftpack import dct, idct
from math import log
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.colors as clr
import numpy as np
# ! pip install tabulate
from tabulate import tabulate

In [ ]:
#exercise 1: image compression testing
def ex1():
    images= ['barn_mountains']# ,'logo','peppers']
    qualities = [75,50,25]
    rates=[]
    for s in images:
        img = Image.open(f"imagens/{s}.bmp")
        rgb_img = img.convert('RGB')
        arr=[s]
        original_size = os.path.getsize(f"imagens/{s}.bmp")
        plt.figure()
        plt.title(f'{s}.bmp')
        plt.imshow(im2)
        for q in qualities:
            rgb_img.save(f"ex1/{s}_{q}.jpeg", quality=q)
            im2 = Image.open(f"ex1/{s}_{q}.jpeg")
            plt.figure()
            plt.title(f'{s}_{q}.jpeg')
            plt.imshow(im2)
            compressed_size = os.path.getsize(f"ex1/{s}_{q}.jpeg")
            comp_rate = "%.1f:1" %(original_size/compressed_size)
            arr.append(comp_rate)
        rates.append(arr)

    # print(46*"_")
    print(tabulate(rates, headers = ['Image\\Quality',"75%","50%","25%"] ))
            # print(f"{s} with quality {q}: compression rate {comp_rate}%")
ex1()

# Results
### barn_mountain.jpeg image

As the quality goes down, there is a somewhat perceptible decrease in detail, especially in zones of the image where there is alternance between light and dark colour within a short range of pixels. This can be explained by the smoothing that is undertaken in JPEG.

Also because of this phenomenon, the contours that the image might have are made less crisp, and some interference between different colours of the respective areas is also verified. This is especcially evident when comparing the original bmp file to the compressed files (mountain-sky interfaces).

### peppers.jpeg image

As is expected, there is also a decrease in detail in this image, as quality decreases. The contours of each vegetable are also affected, but in this case, the smoothing also affects the perceived light reflection coming from the vegetables in general. 

### logo.jpeg image

In this image, which only contains 3 colours, the loss in detail and interference in the various contours is the most evident feature, when analysing different qualities. However, the colour representation within the objects' borders remains accurate, as there is only one colour being represented.

### General Conclusions

Overall, the most apparent quality losses can be noticed in contours and in zones with sharp colour transitions. However, if we consider the compression rates achieved by this format, the storage required is well exploited to represent the images with maximum detail. 

It was also verified that the image with the least features (logo.bmp) had the highest compression rate (81.3:1, 25% quality), whereas the image with most details (barn_mountain.bmp) had the lowest one (29.8:1, 25% quality). This leads to the conclusion that detail makes an image harder to compress.

In [ ]:
def encode(img_name, ds_rate: tuple) -> None:
    img= plt.imread(img_name)
    # plt.figure()
    # plt.imshow(img)
    
    print(img.shape)  # dimensions
    print("image: ",img)
    #R= img[:,:,0] # red channel
    #print(R.shape) 
    print(img.dtype)  # data typr

    img_padded , original_shape = image_padding(img)

    cmRed = color_map('myRed', (0,0,0),(1,0,0))
    cmGreen = color_map('myGreen', (0,0,0),(0,1,0) )
    cmBlue = color_map( 'myBlue', (0,0,0),(0,0,1))
    cmGray = color_map('myGray', (0,0,0),(1,1,1) )
    
    min_cb = (0.5,0.5,0)
    max_cb = (0.5,0.5,1)
    min_cr = (0,0.5,0.5)
    max_cr = (1,0.5,0.5)
    cmChromBlue = color_map('myCb', tuple(min_cb),  tuple(max_cb) )
    cmChromRed = color_map('myCr', tuple(min_cr),  tuple(max_cr) )
    r,g,b = separate_3channels(img_padded)
    chromin_image = rbg2ycbcr(img_padded)
    y, cb,cr = separate_3channels(chromin_image)

    d = {'red':cmRed, 'green': cmGreen, 'blue': cmBlue, 'gray': cmGray,  'chromBlue':cmChromBlue, 'chromRed':cmChromRed }
    e = {'red': r, 'green':g, 'blue':b, 'gray': y, 'chromBlue':cb, 'chromRed':cr}
    # for col in d.keys():
    #     view_image(e[col],d[col])
    print("coisas a acontecer aqui")

    # ds_rate = (4,2,0)
    (y_d,cr_d,cb_d) = ycrcb_downsampling(y,cr,cb, ds_rate)

    
    
    d = {'gray': cmGray, 'chromBlue':cmGray, 'chromRed':cmGray }
    e = { 'gray': y, 'chromBlue':cr_d, 'chromRed':cb_d}

    for col in d.keys():
        view_image(e[col],d[col])

    # return chromin_image, original_shape
    dct_y = dct_channel(y_d)
    dct_cb = dct_channel(cb_d)
    dct_cr = dct_channel(cr_d)
    dcts= {"y":dct_y,"cb":dct_cb,"cr":dct_cr}
    # for name, channel in dcts.items():
    #     fig = plt.figure()
    #     plt.title(f"{name} dct - log(x+0.0001)")
    #     # cax = plt.axes()
    #     # a = np.linspace(np.min(channel),np.max(channel),6)
    #     sh = plt.imshow(np.log(np.abs(channel) + 0.0001))
    #     fig.colorbar(sh)
    #     plt.show(block=False)
       
    return dct_y,dct_cb,dct_cr, original_shape


In [ ]:
def decode(dct_y,dct_cb,dct_cr, ds_ratio, original_shape):
    y_d = idct_channel(dct_y) 
    cb_d = idct_channel(dct_cb) 
    cr_d = idct_channel(dct_cr) 
    dcts= {"y":y_d,"cb":cb_d,"cr":cr_d}
    min_cb = (0.5,0.5,0)
    max_cb = (0.5,0.5,1)
    min_cr = (0,0.5,0.5)
    max_cr = (1,0.5,0.5)
    cmGray = color_map('myGray', (0,0,0),(1,1,1) )
    cmChromBlue = color_map('myCb', tuple(min_cb),  tuple(max_cb) )
    cmChromRed = color_map('myCr', tuple(min_cr),  tuple(max_cr) )
    for name, channel in dcts.items():
        fig = plt.figure()
        plt.title(f"{name} downslampled restored")
        # cax = plt.axes()
        # a = np.linspace(np.min(channel),np.max(channel),6)
        plt.imshow(channel, cmGray)

        plt.show(block=False)
    
    y_u,cr_u, cb_u = ycrcb_upsampling(y_d,cr_d, cb_d, ds_ratio)
    print("cr_u.shape= ",cr_u.shape)
    print("cb_u.shape= ",cb_u.shape)
    d = {'chromBlue':cmChromBlue, 'chromRed':cmChromRed }
    e = {'chromBlue':cb_d, 'chromRed':cr_d}
    # for col in d.keys():
    #     view_image(e[col],d[col])
    d = {'gray': cmGray, 'chromBlue':cmChromBlue, 'chromRed':cmChromRed }
    e = { 'gray': y_u, 'chromBlue':cb_u, 'chromRed':cr_u}
    # for col in d.keys():
    #     view_image(e[col],d[col])
    encoded= join_3channels(y_u,cb_u,cr_u)

    inverse_chromin = ycbcr_to_rgb(encoded)
    plt.figure()
    plt.title('depois de ycbcr e da inversão')
    plt.imshow(inverse_chromin)
    plt.show(block=False)

    img = image_remove_padding(inverse_chromin, original_shape)
    plt.figure()
    plt.title('sem padding')
    plt.imshow(img)
    plt.show(block=False)

    return img
